In [219]:
import pandas as pd
import numpy as np

In [220]:
offices = pd.read_csv('offices.csv', sep=';').set_index('id')

In [221]:
tree = {}
parent = {}

In [222]:
for row in offices['parent_id'].items():
    if not np.isnan(row[1]):
        a, b = int(row[0]), int(row[1])
        if b not in tree:
            tree[b] = []
        tree[b].append(a)
        parent[a] = b

In [223]:
def root_office(office, up = 2):
    for i in range(up):
        if office in parent:
            office = parent[office]
    return office


In [224]:
parent[4]

589